In [1]:
#Script reads in config file in directory to 
#Query data from rtap database
#Review Config File to access specfic database (current database set to 2017; port=5435)

import os
import psycopg2
from configparser import ConfigParser

import pandas as pd
import numpy as np

#bokeh
from bokeh.io import show, output_notebook, push_notebook, curdoc
from bokeh.plotting import figure

from bokeh.layouts import layout, column, row, WidgetBox
from bokeh.models import Panel, Spacer, HoverTool, LogColorMapper, ColumnDataSource, TapTool, BoxSelectTool, CustomJS
from bokeh.models.widgets import Div, Tabs, Paragraph, Dropdown, Button, PreText, Toggle,DataTable, DateFormatter, TableColumn
from bokeh.events import Tap

from bokeh.tile_providers import STAMEN_TERRAIN_RETINA,CARTODBPOSITRON_RETINA

from lxml.html import parse

import bokeh

In [2]:
wdir = r'S:\AdminGroups\ResearchAnalysis\BER\ModelDataViz\TranDataArchive\data'

In [3]:
vds_sensor = pd.read_csv(os.path.join(wdir,'location','vds_sensor_loc_2017.csv'))
il_vds = vds_sensor.loc[vds_sensor['STATE'] == 'IL']

display(il_vds.head(),len(il_vds))

,ID,STATE,LAT,LONG,DIR,ROADNAME
0,IL-TESTTSC-STEVENSON-N-6027,IL,41.76068,-87.88682,NORTH_BOUND,I-55
1,IL-TESTTSC-EDENS-N-1104,IL,41.98256,-87.74818,WEST_BOUND,I-94
2,IL-TESTTSC-EDENS-N-1126,IL,42.13633,-87.78692,WEST_BOUND,I-94
3,IL-TESTTSC-DAN_RYAN_EXPRESS-N-5021,IL,41.86718,-87.64451,WEST_BOUND,I-90
4,IL-TESTTSC-DAN_RYAN_EXPRESS-S-5102,IL,41.87259,-87.64544,EAST_BOUND,I-90


980

In [4]:
#Function to collect and store data column names and types from rtdap database sources
def CollectDataFields(html="db_structure\db_struc.html"):
    DB_DataDict_list = []
    page = parse(html)
    
    #iterate over databases sources
    for db_sources in range(4,8):
        #identify tables for each db source
        tbls = len(page.xpath("/html/body/ul[{0}]".format(db_sources))[0].findall("li"))+1

        #iterate over tables in db sources and collect table column names and types
        for tbl in range(1,tbls):
            tbl_name = page.xpath("/html/body/ul[{0}]/li[{1}]".format(db_sources,tbl))[0].findall("h3")[0].text
            #print(tbl_name)
            rows = page.xpath("/html/body/ul[{0}]/li[{1}]/table".format(db_sources,tbl))[0].findall("tr")
            data = list()
            for row in rows:
                data.append([c.text for c in row.getchildren()])

            for row in data[2:]: 
                DB_DataDict_list.append([tbl_name] + row)

    return pd.DataFrame(DB_DataDict_list,columns=['TableName','ColumnName','DataType','Required','PrimaryKey'])

DB_DataDict = CollectDataFields()
DB_DataDict.to_csv('db_structure\db_fields.csv')
DB_DataDict.head()

,TableName,ColumnName,DataType,Required,PrimaryKey
0,ClarusObservation_YYYY,ObsTypeID,Integer,-,-
1,ClarusObservation_YYYY,ObsTypeName,Text,-,-
2,ClarusObservation_YYYY,ClarusSensorID,Integer,-,-
3,ClarusObservation_YYYY,ClarusStationID,Integer,-,-
4,ClarusObservation_YYYY,ClarusSensorIndex,Integer,-,-


In [5]:
#read in config file containing connection parameters
def config(filename='rtdap_2017.ini',section='postgresql'):
    #create parser
    parser = ConfigParser()
    #read config file
    parser.read_file(open(filename))
    
    #get section, default to postgresql
    db={}
    if parser.has_section(section):
        params= parser.items(section)
        for param in params:
            #iterate over items in config file
            #to inject dbname, host, user, and password
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in {1} file'.format(section,filename))
        
    
    return db

#establish connection to database
def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        #read in configuration parameters
        params = config()
        
        #connect to PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)
        
        #create cursor
        cur = conn.cursor()
        
    #execute a statement
        print('PostgreSQL database version:')
        cur.execute('SELECT version()')

        #display the PosgreSQL database server version
        db_version = cur.fetchone()
        print(db_version)

        #close the communication with the PostgreSQL 
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')
 

 
if __name__ == '__main__':
    connect()
    

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 9.1.18, compiled by Visual C++ build 1500, 64-bit',)
Database connection closed.


In [7]:
#return all results (limit first 10)
def select_table(data_table):
    """ query records from db table"""
    conn = None
    try:
        params = config()
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        cur.execute('''
            select "FieldDeviceID",
            "Occupancy",
            "Volume",
            "Speed",
            extract(isodow from "LocationTimeStamp") AS "dow",
            extract(year from "LocationTimeStamp") AS "year",
            extract(month from "LocationTimeStamp") AS "month",
            extract(doy from "LocationTimeStamp") AS "doy",
            extract(hour from "LocationTimeStamp") AS "hour"
            from "''' + data_table + '''"
            where extract(isodow from "LocationTimeStamp") <  %s AND "DeviceStatus" = 'OPERATIONAL' 
            AND LEFT("OwningAgencyID",2) = 'IL' AND "FieldDeviceID" = 'IL-TESTTSC-STEVENSON-N-6027'
            limit 5'''% ("'6'"))
        result = [row for row in cur.fetchall()]
        names = [name[0] for name in cur.description]
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            
    return pd.DataFrame(result,columns=names)

In [8]:
#return results
vds_detail = select_table("GatewayVDSDetail_2017")
vds_detail.head()

,FieldDeviceID,Occupancy,Volume,Speed,dow,year,month,doy,hour
0,IL-TESTTSC-STEVENSON-N-6027,4.632143,528.0,26.006962,1.0,2016.0,5.0,130.0,10.0
1,IL-TESTTSC-STEVENSON-N-6027,65.707146,324.0,1.397325,1.0,2016.0,5.0,130.0,10.0
2,IL-TESTTSC-STEVENSON-N-6027,65.257140,180.0,0.782251,1.0,2016.0,5.0,130.0,10.0
3,IL-TESTTSC-STEVENSON-N-6027,14.307143,1440.0,23.616318,1.0,2016.0,5.0,130.0,10.0
4,IL-TESTTSC-STEVENSON-N-6027,16.975000,1248.0,16.929893,1.0,2016.0,5.0,130.0,10.0


In [16]:
config_files = [('rtdap_2017.ini','GatewayVDSDetail_2017'),
                ('rtdap_2016.ini','GatewayVDSDetail_2016'),
                ('rtdap_2013_2015.ini','GatewayVDSDetail_2015'),
                ('rtdap_2013_2015.ini','GatewayVDSDetail_2014'),
                ('rtdap_2013_2015.ini','GatewayVDSDetail_2013')]

def pull_summary(config_file,data_table):
    def config(filename=config_file,section='postgresql'):
        #create parser
        parser = ConfigParser()
        #read config file
        parser.read_file(open(filename))

        #get section, default to postgresql
        db={}
        if parser.has_section(section):
            params= parser.items(section)
            for param in params:
                #iterate over items in config file
                #to inject dbname, host, user, and password
                db[param[0]] = param[1]
        else:
            raise Exception('Section {0} not found in {1} file'.format(section,filename))


        return db

    #establish connection to database
    def connect():
        """ Connect to the PostgreSQL database server """
        conn = None
        try:
            #read in configuration parameters
            params = config()

            #connect to PostgreSQL server
            print('Connecting to the PostgreSQL database...')
            conn = psycopg2.connect(**params)

            #create cursor
            cur = conn.cursor()

        #execute a statement
            print('PostgreSQL database version:')
            cur.execute('SELECT version()')

            #display the PosgreSQL database server version
            db_version = cur.fetchone()
            print(db_version)

            #close the communication with the PostgreSQL 
            cur.close()
        except (Exception, psycopg2.DatabaseError) as error:
            print(error)
        finally:
            if conn is not None:
                conn.close()
                print('Database connection closed.')

    def select_table(data_table):
        """ query records from db table"""
        conn = None
        try:
            params = config()
            conn = psycopg2.connect(**params)
            cur = conn.cursor()
            cur.execute('''
                select "FieldDeviceID",
                "Occupancy",
                "Volume",
                "Speed",
                extract(isodow from "LocationTimeStamp") AS "dow",
                extract(year from "LocationTimeStamp") AS "year",
                extract(month from "LocationTimeStamp") AS "month",
                extract(doy from "LocationTimeStamp") AS "doy",
                extract(hour from "LocationTimeStamp") AS "hour"
                from "''' + data_table + '''"
                where extract(isodow from "LocationTimeStamp") <  %s AND "DeviceStatus" = 'OPERATIONAL' 
                AND LEFT("OwningAgencyID",2) = 'IL' AND "FieldDeviceID" = 'IL-TESTTSC-STEVENSON-N-6027'
                limit 5'''% ("'6'"))
            result = [row for row in cur.fetchall()]
            names = [name[0] for name in cur.description]
            cur.close()
        except (Exception, psycopg2.DatabaseError) as error:
            print(error)
        finally:
            if conn is not None:
                conn.close()

        return pd.DataFrame(result,columns=names)


    #return all results (limit first 10)
    def select_vds(data_table):
        """ query records from db table"""
        conn = None
        try:
            params = config()
            conn = psycopg2.connect(**params)
            cur = conn.cursor()
            cur.execute('''
                select "FieldDeviceID",
                AVG("Occupancy"),
                AVG("Volume"),
                AVG("Speed"),
                extract(isodow from "LocationTimeStamp") AS "dow",
                extract(year from "LocationTimeStamp") AS "year",
                extract(month from "LocationTimeStamp") AS "month",
                extract(doy from "LocationTimeStamp") AS "doy",
                extract(hour from "LocationTimeStamp") AS "hour"
                from "''' + data_table + '''"
                where extract(isodow from "LocationTimeStamp") <  %s AND "DeviceStatus" = 'OPERATIONAL' 
                AND LEFT("FieldDeviceID",2) = 'IL'
                group by "FieldDeviceID", 
                extract(year from "LocationTimeStamp"),
                extract(month from "LocationTimeStamp"),
                extract(isodow from "LocationTimeStamp"),
                extract(doy from "LocationTimeStamp"),
                extract(hour from "LocationTimeStamp")
                ''' % ("'6'"))
            result = [row for row in cur.fetchall()]
            names = [name[0] for name in cur.description]
            cur.close()
        except (Exception, psycopg2.DatabaseError) as error:
            print(error)
        finally:
            if conn is not None:
                conn.close()

        return pd.DataFrame(result,columns=names)
    
    
    if __name__ == '__main__':
        connect()
    
    try:
        vds_table = select_vds(data_table)
        vds_table.to_csv(os.path.join(wdir,data_table+'.csv'),index=False)
    except:
        pass

    return display(vds_table.head())
    

In [17]:
for i in config_files:
    pull_summary(i[0],i[1])

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 9.1.18, compiled by Visual C++ build 1500, 64-bit',)
Database connection closed.


,FieldDeviceID,avg,avg,avg,dow,year,month,doy,hour
0,IL-TESTTSC-LAKE_SHORE_DRIVE_NORTH-S-10211,12.974649,847.862669,NaN,3.0,2017.0,4.0,95.0,12.0
1,IL-TESTTSC-KENNEDY-W-2123,12.744540,1015.308605,NaN,1.0,2016.0,5.0,130.0,10.0
2,IL-TESTTSC-KINGERY-E-8107,6.558887,672.784008,NaN,1.0,2016.0,5.0,130.0,10.0
3,IL-TESTTSC-KENNEDY-E-2025,16.765638,1043.984521,NaN,1.0,2016.0,5.0,130.0,10.0
4,IL-TESTTSC-EISENHOWER-E-3009,16.390939,1272.354089,NaN,1.0,2016.0,5.0,130.0,10.0


Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 9.1.18, compiled by Visual C++ build 1500, 64-bit',)
Database connection closed.


,FieldDeviceID,avg,avg,avg,dow,year,month,doy,hour
0,IL-IDOTD2-I-74 NB from 12th Ave to 7th Ave,2.483685,353.882414,28.479043,2.0,2016.0,9.0,250.0,20.0
1,IL-IDOTD2-I-74 NB from 1st Ave to Toll Pla,4.017385,314.632324,13.555334,2.0,2016.0,9.0,250.0,20.0
2,IL-IDOTD2-I-74 NB from 4th Ave to 1st Ave,3.016636,496.279117,24.431017,2.0,2016.0,9.0,250.0,20.0
3,IL-IDOTD2-I-74 NB from 7th Ave to 4th Ave,2.777585,354.420885,28.157265,2.0,2016.0,9.0,250.0,20.0
4,IL-IDOTD2-I-74 NB from 7th Ave to 4th Ave_RAMP,0.256237,17.301783,17.256039,2.0,2016.0,9.0,250.0,20.0


Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 9.1.18, compiled by Visual C++ build 1500, 64-bit',)
Database connection closed.


,FieldDeviceID,avg,avg,avg,dow,year,month,doy,hour
0,IL-TESTTSC-KINGERY-E-8106,9.232544,392.583333,28.619301,4.0,2014.0,12.0,345.0,9.0
1,IL-TESTTSC-KINGERY-W-8003,0.000000,0.000000,NaN,4.0,2014.0,12.0,345.0,21.0
2,IL-TESTTSC-I_80-E-35043,13.066667,240.000000,29.354195,1.0,2014.0,12.0,349.0,8.0
3,IL-TESTTSC-I_80-W-35129,13.585925,450.571429,31.654812,1.0,2014.0,12.0,335.0,20.0
4,IL-TESTTSC-I_80-W-35123,19.149998,405.000000,28.975230,4.0,2014.0,12.0,338.0,4.0


Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 9.1.18, compiled by Visual C++ build 1500, 64-bit',)
Database connection closed.


,FieldDeviceID,avg,avg,avg,dow,year,month,doy,hour
0,IL-TESTTSC-I_80-E-35039,7.466667,90.000000,29.372830,1.0,2014.0,11.0,328.0,0.0
1,IL-TESTTSC-I_80-W-35132,7.268445,204.000000,31.031265,4.0,2014.0,11.0,310.0,18.0
2,IL-TESTTSC-I_80-W-35129,6.800001,165.000000,33.100350,1.0,2014.0,11.0,314.0,1.0
3,IL-TESTTSC-I_80-E-35039,1.438148,15.333333,25.645315,3.0,2014.0,11.0,316.0,2.0
4,IL-TESTTSC-KINGERY-E-8107,3.136559,167.166667,26.679600,4.0,2014.0,11.0,331.0,0.0


Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 9.1.18, compiled by Visual C++ build 1500, 64-bit',)
Database connection closed.


,FieldDeviceID,avg,avg,avg,dow,year,month,doy,hour
0,IL-TESTTSC-I_290-W-4134,3.731839,269.069363,NaN,4.0,2008.0,10.0,290.0,22.0
1,IL-TESTTSC-BISHOP_FORD-N-47005,0.000000,0.000000,NaN,4.0,2013.0,8.0,234.0,4.0
2,IL-TESTTSC-I_80-W-35128,7.181102,69.744526,27.732723,2.0,2013.0,10.0,302.0,16.0
3,IL-TESTTSC-I_80-E-35071,5.032425,140.870123,NaN,4.0,2008.0,10.0,290.0,22.0
4,IL-TESTTSC-KENNEDY-W-2131,10.394179,364.340905,NaN,4.0,2008.0,10.0,290.0,22.0


In [10]:
config_files = [('rtdap_2018.ini','GatewayVDSDetail_2018'),
                ('rtdap_2017.ini','GatewayVDSDetail_2017'),
                ('rtdap_2016.ini','GatewayVDSDetail_2016'),
                ('rtdap_2013_2015.ini','GatewayVDSDetail_2015'),
                ('rtdap_2013_2015.ini','GatewayVDSDetail_2014'),
                ('rtdap_2013_2015.ini','GatewayVDSDetail_2013')]

vds_table = pd.DataFrame()

#format and join files
for i in config_files:
    df = pd.read_csv(os.path.join(wdir,i[1]+'.csv'))
    df.columns = ['FieldDeviceID','avgOccupancy','avgVolume','avgSpeed','dow','year','month','day','hour']
    vds_table = vds_table.append(df)
    

vds_table = vds_table.fillna(99999)    
vds_table.loc[(vds_table['avgSpeed']!=99999) &\
              (vds_table['avgVolume']!=99999) &\
              (vds_table['avgSpeed']!=99999)].groupby(['year']).agg({'FieldDeviceID':'count',
                                                                     'avgOccupancy':np.mean,
                                                                     'avgVolume':np.mean,
                                                                     'avgSpeed':np.mean})

,FieldDeviceID,avgOccupancy,avgVolume,avgSpeed
year,,,,
2008.0,27,9.271831,287.619924,24.840301
2010.0,1,4.610704,206.598542,38.921677
2011.0,8,8.396640,303.714739,27.592211
2013.0,527,4.801068,381.899976,31.230679
2014.0,31299,6.743934,206.018035,29.938845
2015.0,16119,3.060707,284.022017,27.252030
2016.0,45445,4.043486,353.378949,28.590456
2017.0,100,6.671231,615.993532,25.245579
2018.0,65,6.434568,582.326637,24.406003


In [9]:
vds_table.to_csv(r'C:\Users\bross\Documents\GitHub\rtdap\rtdap_prototype\data\vds_table.csv')